# Set up


In [1]:
#connect to gdrive
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive
#%cd ../..
!pwd

Mounted at /content/gdrive
/content/gdrive/MyDrive
/content/gdrive/MyDrive


In [2]:
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"yuzhongfinally","key":"da5ea7094c4adb8f29251a2e3b240da7"}
with open('/content/gdrive/MyDrive/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!chmod 600 /content/gdrive/MyDrive/.kaggle/kaggle.json
!cp /content/gdrive/MyDrive/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content/gdrive/MyDrive/

mkdir: cannot create directory ‘.kaggle’: File exists
cp: cannot create regular file '/root/.kaggle/': Not a directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [3]:
!kaggle competitions download -c 11785-spring2021-hw1p2
%cd competitions/11785-spring2021-hw1p2/
!ls

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
/content/gdrive/MyDrive/competitions/11785-spring2021-hw1p2
dev_labels.npy	    model_context_40		  sample.csv.zip
dev_labels.npy.zip  model_lr_0.1		  tb_logs
dev.npy		    model_lr_bn_0.1		  test.npy
dev.npy.zip	    ngrok			  test.npy.zip
model_context_15    ngrok-stable-linux-amd64.zip  train_labels.npy
model_context_20    phonemes.txt		  train.npy
model_context_25    propose_prediction.csv	  train.npy.zip
model_context_30    sample.csv


In [4]:
%cd competitions/11785-spring2021-hw1p2/
"""
!unzip dev.npy.zip
!unzip dev_labels.npy.zip
!unzip sample.csv.zip

!pwd
!unzip train.npy.zip
"""

[Errno 2] No such file or directory: 'competitions/11785-spring2021-hw1p2/'
/content/gdrive/MyDrive/competitions/11785-spring2021-hw1p2


'\n!unzip dev.npy.zip\n!unzip dev_labels.npy.zip\n!unzip sample.csv.zip\n\n!pwd\n!unzip train.npy.zip\n'

# Pre-processing


In [50]:
import torch
import numpy as np
import torch.nn as nn

In [51]:
class Dataset_train(torch.utils.data.Dataset):
  def __init__(self, X, Y, offset , context ):
    self.X = X
    self.Y = Y

    index_map_X = []

    for i, x in enumerate(X):
      for j, xx in enumerate(x):
        index_map_X.append((i, j))

    self.index_map = index_map_X
        
    self.length = len(self.index_map)
    self.context = context
    self.offset = offset
        
    for i, x in enumerate(self.X):
        self.X[i] = np.pad(x, ((context, context), (0, 0)), 'constant', constant_values=0)
            
        
  def __len__(self):
      
      return self.length
    
  def __getitem__(self, index):
      
      i, j = self.index_map[index]
      
      start_j = j + self.offset - self.context
      
      end_j = j + self.offset + self.context + 1
      
      xx = self.X[i][start_j:end_j,:]

      yy = self.Y[i][j]
     
      return xx ,yy  


In [52]:
def dataloader(data, label, offset, context, train_loader_args):
  dataset = Dataset_train(data, label, offset, context)
  dataloader = torch.utils.data.DataLoader(dataset,**train_loader_args)
                                        # collate_fn=Dataset_train.collate_fn)
  return dataloader

# Training

In [53]:
class MLP(torch.nn.Module):
    # you can use the layer sizes as initialization arguments if you want to
    def __init__(self,size_list):
        super(MLP, self).__init__()
        layers = []
        self.size_list = size_list
        for i in range(len(size_list) - 2):
          
          layers.append(nn.Linear(size_list[i], size_list[i+1]))
          layers.append(torch.nn.BatchNorm1d(size_list[i+1]))
          layers.append(nn.ReLU())
          
        layers.append(nn.Linear(size_list[-2], size_list[-1]))
        self.net = nn.Sequential(*layers)

    def forward(self, input_val):
 
      return self.net(input_val)


In [54]:
from torch.optim.lr_scheduler import StepLR
def train_parameters(model,criterion, iter, lr, data):

  model.train()
  optimizer = torch.optim.SGD(model.parameters(),lr = lr, momentum = 0.9, nesterov = True)
  #optimizer = torch.optim.Adam(model.parameters(), lr = lr, betas= (0.9, 0.99))
  scheduler = StepLR(optimizer, step_size = 1, gamma = 0.1)
  
  model = model.to(device= 'cuda')

  for i in range(iter):
    print(f'Epoch: {i} ')
    
    
    for X_train, y_train in data:
      optimizer.zero_grad()

      X_train = X_train.to(device='cuda')
      y_train = y_train.to(device='cuda')

    
      X_train = X_train.view(X_train.size(0), -1)
      y_hat = model(X_train.float())

      loss = criterion(y_hat, y_train.long())
      loss.backward()
      optimizer.step()
      

    scheduler.step()
  

# Validating

In [55]:
from sklearn.metrics import accuracy_score

def model_val(net, dataloader_val):
  with torch.no_grad():
    net.eval()

    total_preds = []
    truth = []
    
    for i,  (X_test, y_test) in enumerate(dataloader_val):
      print(i)
      X_test = X_test.to(device='cuda')
      X_test = X_test.view(X_test.size(0), -1)
      y_test = y_test.to(device='cuda')
       
      
      #output
      output = net(X_test.float())
      loss = criterion(output, y_test.long())

      output = output.cpu()
      y_test = y_test.cpu()
      

      output = output.detach().numpy()
      true = y_test.numpy()
      
      output = np.argmax(output,axis = 1)
      
      #reshape
      
      true = true.reshape(-1,1)
      output = output.reshape(-1,1)

      total_preds.append(output)
      truth.append(true)
    
 
  print('finished')
  truth, total_preds = np.vstack(truth), np.vstack(total_preds)
  acc = accuracy_score(truth, total_preds)
  print(acc)
  return acc, loss.item()
   

# run loop to find best parameter

In [56]:
#tuning
train = np.load('train.npy', allow_pickle = True)
train_labels = np.load('train_labels.npy',allow_pickle = True)
context = 25
offset = 25
batch_size = 64
num_workers = 0
train_loader_args = dict(shuffle = True, batch_size = batch_size, num_workers = num_workers, drop_last = True) 
dataloader_train = dataloader(train,train_labels, offset, context, train_loader_args )

del train
del train_labels
#!unzip dev.npy.zip
#!unzip dev_labels.npy.zip
dev = np.load('dev.npy', allow_pickle = True)
dev_labels = np.load('dev_labels.npy',allow_pickle = True)
dev_loader_args = dict(shuffle = True, batch_size = batch_size, num_workers =0, drop_last = True)
dataloader_val  = dataloader(dev,dev_labels, offset, context, dev_loader_args)
del dev
del dev_labels



In [57]:
# against learning rate
lr = [0.1]
#build the network

context = 25
input = (2 * int(context)+ 1) * 40
output = 71

hidden_lyr =[input,900,900, 450, 225,output]
criterion = nn.CrossEntropyLoss()
epoch =5
report_lr = {}
for l in lr:
  model = MLP(hidden_lyr)
  print(model)
  accu_loss = train_parameters(model,criterion, epoch, l, dataloader_train)
  case_accuracy = model_val(model, dataloader_val)
  report_lr[l] = case_accuracy

  if case_accuracy > 0.75:
    torchname = "model_lr_bn_"+str(l)
    torch.save(model.state_dict(), torchname)

  print(report_lr)


MLP(
  (net): Sequential(
    (0): Linear(in_features=2040, out_features=900, bias=True)
    (1): BatchNorm1d(900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=900, out_features=900, bias=True)
    (4): BatchNorm1d(900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=900, out_features=450, bias=True)
    (7): BatchNorm1d(450, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=450, out_features=225, bias=True)
    (10): BatchNorm1d(225, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=225, out_features=71, bias=True)
  )
)
Epoch: 0 
Epoch: 1 
Epoch: 2 
Epoch: 3 


KeyboardInterrupt: ignored

In [ ]:
#experiment: context size 
"""
context = [ 20, 25, 30, 40, 50]
#input = (2 * int(context)+ 1) * 40
hidden1 = 1024
hidden2 = 1024
hidden3 = 512
hidden4 = 256
output = 71


for c in context:

  criterion = nn.CrossEntropyLoss()
  epoch = 5
  lr = 0.1
  report_context = {}

  #decide context
  input = (2 * int(c)+ 1) * 40
  model = MLP([input, hidden1, hidden2 ,hidden3, hidden4,output])
  print(model)
  print('training')

  #read files
  train = np.load('train.npy', allow_pickle = True)
  train_labels = np.load('train_labels.npy',allow_pickle = True)
  train_loader_args = dict(shuffle = True, batch_size = batch_size, num_workers = num_workers, drop_last = True) 
  dataloader_train = dataloader(train,train_labels, c, c, train_loader_args )
  del train
  del train_labels
  #!unzip dev.npy.zip
  #!unzip dev_labels.npy.zip
  dev = np.load('dev.npy', allow_pickle = True)
  dev_labels = np.load('dev_labels.npy',allow_pickle = True)
  dev_loader_args = dict(shuffle = True, batch_size = batch_size, num_workers =0, drop_last = True)
  dataloader_val  = dataloader(dev,dev_labels, c, c, dev_loader_args)
  del dev
  del dev_labels


  accu_loss = train_parameters(model,criterion, epoch, lr, dataloader_train)
  case_accuracy = model_val(model, dataloader_val)
  report_context[c] = case_accuracy
  print(report_context)
  torchname = "model_context_"+str(c)
  torch.save(model.state_dict(), torchname)
"""

In [ ]:
"""
#test: against different layers
#build the network
input = (2 * int(context)+ 1) * 40
hidden1 = 1024
hidden2 = 1024
hidden3 = 512
output = 71

hidden_lyr =[#[input, 1024, 1024, 512, output],
             [input, 1024, 512, output],[input, 1024, 1024, 512, 256, output]]
criterion = nn.CrossEntropyLoss()
epoch = 5
lr = 0.0001
#report_lyr = {}
for h in hidden_lyr:
  model = MLP(h)
  print(model)
  accu_loss = train_parameters(model,criterion, epoch, lr, dataloader_train)
  case_accuracy = model_val(model, dataloader_val)
  report_lyr[str(h)] = case_accuracy
  print(report_lyr)
  
"""

# add Test set and submit

In [58]:

#!unzip test.npy.zip

test = np.load('test.npy', allow_pickle= True)

In [59]:
class load_test(torch.utils.data.Dataset):
    
    def __init__(self, X, offset, context):
        
        self.X = X
        
        index_map_X = []
        
        for i, x in enumerate(X):
            for j, xx in enumerate(x):
                index_pair_X = (i, j)
                index_map_X.append(index_pair_X)
                
        self.index_map = index_map_X
        
        self.length = len(self.index_map)
        
        self.context = context
        self.offset = offset

        for i, x in enumerate(self.X):
            self.X[i] = np.pad(x, ((context, context), (0, 0)), 'constant', constant_values=0)
        
    def __len__(self):

        return self.length
    
    def __getitem__(self, index):

        i, j = self.index_map[index]
        
        start_j = j + self.offset - self.context

        end_j = j + self.offset + self.context + 1

        xx = self.X[i][start_j:end_j,:]

        return xx


In [60]:

test_loader_args = dict(shuffle = False, batch_size = batch_size, num_workers = num_workers, pin_memory =  True) 

dataset = load_test(test,offset, context )
dataloader_test = torch.utils.data.DataLoader(dataset,**test_loader_args)



In [61]:

preds_total = []
model.cuda()
with torch.no_grad():
  for X_test in dataloader_test:
    X_test = X_test.to(device='cuda')
    y_hat = model(X_test.view(X_test.size(0), -1).float())
    
    y_hat = y_hat.cpu()
    y_hat = y_hat.detach().numpy()
    
    preds = np.argmax(y_hat, axis = 1)
    preds = preds.reshape(-1,1)
    preds_total.append(preds)


Streaming output truncated to the last 5000 lines.
 [25]
 [25]
 [25]
 [66]
 [66]
 [66]
 [66]
 [66]
 [66]
 [66]
 [27]
 [27]
 [27]
 [27]
 [27]
 [27]
 [27]
 [27]
 [27]
 [27]
 [69]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [40]
 [31]
 [31]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [31]
 [ 0]
 [40]
 [40]
 [40]
 [40]
 [40]
 [40]]
[[39]
 [45]
 [45]
 [ 0]
 [ 0]
 [58]
 [58]
 [58]
 [20]
 [20]
 [ 0]
 [ 0]
 [20]
 [20]
 [ 0]
 [ 8]
 [ 8]
 [58]
 [ 0]
 [ 0]
 [ 0]
 [21]
 [21]
 [43]
 [58]
 [35]
 [35]
 [37]
 [56]
 [64]
 [64]
 [61]
 [64]
 [64]
 [16]
 [67]
 [67]
 [67]
 [67]
 [67]
 [67]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [69]
 [69]
 [69]
 [69]
 [69]
 [69]
 [58]
 [23]
 [23]
 [23]
 [23]
 [25]
 [25]
 [25]
 [ 7]
 [ 6]
 [55]
 [55]
 [55]]
[[ 8]
 [ 8]
 [55]
 [55]
 [58]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [46]
 [46]
 [46]
 [68]
 [37]
 [37]
 [68]
 [68]
 [37]
 [55]
 [55]
 [55]
 [55]
 [55]
 [55]
 [27]
 [27]
 [27]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [56]
 [56]
 [69]
 [69]
 [69]
 [69]
 [56]
 [56]

In [62]:
preds_final = []
for i in preds_total:
  for t in i:
    preds_final.append(int(t))


In [ ]:
import pandas as pd
df = pd.DataFrame({'id':np.arange(len(preds_final)), "label":preds_final})
df.to_csv(r"propose_prediction.csv", index=False)

In [ ]:
!kaggle competitions submit -c 11785-spring2021-hw1p2 -f propose_prediction.csv -m "Message"

100% 15.5M/15.5M [00:05<00:00, 3.23MB/s]
Successfully submitted to 11785-Spring2021-Hw1P2